In [1]:
import pandas as pd
import sqlite3

In [2]:
# Read dana's files
community_benefit_df = pd.read_csv("datasets\Owned Club Community Benefits Review 20181220 V8 Preprocessed.csv",dtype=str)
print("Before dropping duplicates: ")
print(len(community_benefit_df.index))

community_benefit_df.drop_duplicates(keep='first', inplace=True)
print("After dropping duplicates: ")
print(len(community_benefit_df.index))

Before dropping duplicates: 
1499
After dropping duplicates: 
1497


In [3]:
#Reading Marekt Ids
market_id_df = pd.read_excel("datasets\MarketNameMapping.xlsx", sheet_name=0,dtype=str)
#market_id_df.head()
market_id_df['Market_Name_Dana'] = market_id_df['Market_Name_Dana'].apply(lambda x: x.strip().upper())
market_id_df['Market_Name_Dana'] = market_id_df['Market_Name_Dana'].apply(lambda x: ' '.join(x.split()))
market_id_df = market_id_df.drop_duplicates()
market_id_df = market_id_df.drop_duplicates(keep=False)
print(len(market_id_df.index))

62


In [4]:
# Reading Community Ids
community_id_df = pd.read_excel("datasets\Community Name Mapping.xlsx", sheet_name=0,dtype=str)
community_id_df['Community_Name_Dana'] = community_id_df['Community_Name_Dana'].apply(lambda x: x.strip().upper())
community_id_df['Community_Name_Dana'] = community_id_df['Community_Name_Dana'].apply(lambda x: ' '.join(x.split()))
#community_id_df.head()
community_id_df = community_id_df.drop_duplicates()

In [5]:
# Reading club ids
#entity_code_df = pd.read_excel("datasets//full club list_20181218.xlsx", sheet_name=0,dtype=str)
entity_code_df = pd.read_excel("datasets//full club list_20190130.xlsx", sheet_name=0,dtype=str)
# Make entity code 5 digits
entity_code_df = entity_code_df[['Community','Market', 'Club','Entity_Code']]
entity_code_df['Entity_Code'] = entity_code_df['Entity_Code'].apply(lambda x: str(x).zfill(5))

#Make club name to upper case,trim and removing consecutive space
entity_code_df['Club'] = entity_code_df['Club'].apply(lambda x: x.strip().upper())
entity_code_df['Club'] = entity_code_df['Club'].apply(lambda x: ' '.join(x.split()))

entity_code_df['Community'] = entity_code_df['Community'].apply(lambda x: x.strip().upper())
entity_code_df['Community'] = entity_code_df['Community'].apply(lambda x: ' '.join(x.split()))

entity_code_df['Market'] = entity_code_df['Market'].apply(lambda x: x.strip().upper())
entity_code_df['Market'] = entity_code_df['Market'].apply(lambda x: ' '.join(x.split()))

entity_code_df = entity_code_df.drop_duplicates()
#entity_code_df.head()

In [6]:
%%time
conn = sqlite3.connect("CommunityBenefit.db")
cur = conn.cursor()
community_benefit_df.to_sql("community_benefit", conn, if_exists="replace")
community_id_df.to_sql("community_id", conn, if_exists="replace")
market_id_df.to_sql("market_id", conn, if_exists="replace")
entity_code_df.to_sql("entity_code", conn, if_exists="replace")

C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


Wall time: 335 ms


In [7]:
# query for merging dana's file with the id file
# we exclude those rows of dana's file that does associated with a community id or, club id

query = """select distinct        
            community_benefit.Community,
            community_id.Community_ID as community_id,
            community_benefit.Market,
            market_id.Market_ID_BF as market_id,
            community_benefit.'Club Type',
            community_benefit.Club,
            entity_code.Entity_Code as entity_code,
            community_benefit.'Benefit Tier',
            community_benefit.'Cart Fee Rate',
            community_benefit.'Suggested Tier Rate 2019',
            community_benefit.'Golf Benefit',
            community_benefit.'Availability',
            community_benefit.'Advance',
            community_benefit.'Golf Amenity Restrictions',
            community_benefit.'Dining Benefit',
            community_benefit.'Dining Availability',
            community_benefit.'Dining Restrictions',
            community_benefit.'Tennis Benefit',
            community_benefit.'Tennis Availability',
            community_benefit.'Tennis Advance',
            community_benefit.'Athletic Benefit',
            community_benefit.'Athletic Availability',
            community_benefit.'Athletic Advance',
            community_benefit.'Pool Benefit',
            community_benefit.'Pool Availability',
            community_benefit.'Pool Advance',
            community_benefit.Notes
            from community_benefit
            left join community_id on community_id.Community_Name_Dana = community_benefit.Community
            left join market_id on market_id.Market_Name_Dana = community_benefit.Market
            left join entity_code on entity_code.Club = community_benefit.Club and entity_code.Community=community_id.Community_Name_Dana and entity_code.Market=community_benefit.Market
            """

In [8]:
rs_benefit = pd.read_sql_query(query, conn)
#rs_benefit.head()
print(len(rs_benefit.index))

1497


In [9]:
print(len(rs_benefit.index))
rs_benefit.to_csv("output/intermediate/Danas_File_ExtendedWithIds.csv",index=False)

1497


In [10]:
conn.execute("DROP TABLE community_benefit")
conn.execute("DROP TABLE community_id")
conn.execute("DROP TABLE market_id")
conn.execute("DROP TABLE entity_code")
cur.close()
conn.close()